# AlexNet
## Key Elements
#### Objective:
Demonstrate the effectiveness of deep convolutional neural networks on image classification tasks, specifically the ImageNet dataset.

#### Innovations:
- Use of ReLU activation for faster training
- Dropout to mitigate overfitting
- Data Augmentation (random crops, flips)

## Architecture
- Input: 224X224 RGB images.
- Conv Layers: Five convolutional layers, some with max-pooling
- Dense Layers: Three fully connected layers, with dropout applied before the final layer
- Output: 1000 softmax units for ImageNet classification

In [1]:
# importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
class AlexNet(nn.Module):
  def __init__(self, num_classes=10):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
        # Layer 1
        nn.Conv2d(3,  96, kernel_size=11, stride=4, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # Layer 2
        nn.Conv2d(96, 256, kernel_size=5, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        #Layer 3
        nn.Conv2d(256, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        # Layer 4
        nn.Conv2d(384, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        # Layer 5
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )

  def forward(self,x):
    x = self.features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

In [3]:
# training loop
def train_model(model,dataloader, criterion, optimizer, epochs=5, device="cuda"):
  model.to(device)
  for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")

In [5]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Using CIFAR-10 dataset for demonstration
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)


model = AlexNet(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


train_model(model, train_loader, criterion, optimizer, epochs=5)


Files already downloaded and verified
Epoch 1/5, Loss: 110951.56030320148
Epoch 2/5, Loss: 2.3031692139956417
Epoch 3/5, Loss: 2.303185224533081
Epoch 4/5, Loss: 2.303105871288144
Epoch 5/5, Loss: 2.303004517847178
